In [2]:
!pip install -q mediapipe

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
DATA_PATH=os.path.join('MP_DATA')
actions=np.array(['hello','thanks','iloveyou','victory','father','mother','help','yes','no'])
no_sequences=30
sequence_length=30

In [6]:
label_map = {label:num for num, label in enumerate(actions)}

In [7]:
label_map

{'hello': 0,
 'thanks': 1,
 'iloveyou': 2,
 'victory': 3,
 'father': 4,
 'mother': 5,
 'help': 6,
 'yes': 7,
 'no': 8}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
X = np.array(sequences) 

In [13]:
X

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.46096045e-01,  6.33533001e-01, -7.36023113e-02],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.86922723e-01,  5.04557371e-01, -3.20381075e-02],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e

In [15]:
y = to_categorical(labels).astype(int) 

In [16]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # split the data to train and test 

In [18]:
y_test.shape #( no of tested images, words )

(27, 9)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers import Flatten

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()    # linear stack of layers.
model.add(Flatten(input_shape=(30,126)))
model.add(Dense(64,  activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(128,  activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2)) # add dropout layer
model.add(Dense(9, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, mode='min', verbose=1)
model.fit(X_train,y_train,epochs=2000, batch_size=32,callbacks=[early_stopping],validation_data=(X_test, y_test))

In [ ]:
model.summary()

In [ ]:
model.save('actionmodel.h5') 

In [ ]:
model.load_weights('actionmodel.h5')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)